# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [3]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import time

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [4]:
weather_data = pd.read_csv("../Weather Py/output_data/cities.csv")
weather_data


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Punta Arenas,-53.1500,-70.9167,53.71,47,40,32.21,CL,1643579868
1,Mataura,-46.1927,168.8643,68.49,68,48,0.47,NZ,1643580059
2,Otane,-39.8833,176.6333,68.38,60,100,3.98,NZ,1643580059
3,Taltal,-25.4000,-70.4833,67.98,72,29,7.43,CL,1643579989
4,Ribeira Grande,38.5167,-28.7000,59.13,73,47,26.37,PT,1643580060
...,...,...,...,...,...,...,...,...,...
572,Kang,-23.6752,22.7876,68.83,82,99,14.12,BW,1643580433
573,Alofi,-19.0595,-169.9187,73.87,87,100,8.57,NU,1643580435
574,Teguise,29.0605,-13.5640,62.82,39,0,9.22,ES,1643580435
575,Katsuura,35.1333,140.3000,38.19,52,100,11.48,JP,1643580436


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
# gmaps
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = weather_data[["Lat", "Lng"]]

# Store Humidity in humidity
humidity = weather_data["Humidity"]

In [6]:
# Plot Heatmap
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)
max_intensity = np.max(humidity)

# Add heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=100, point_radius=3)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
# Narrow down the cities with wind speed < 10 mph, cloudiness = 0 and max temp between 60 and 80
narrow_city_df = weather_data.loc[(weather_data["Wind Speed"] <= 10) & (weather_data["Cloudiness"] == 0) & \
                                   (weather_data["Max Temp"] >= 70) & (weather_data["Max Temp"] <= 80)].dropna()

narrow_city_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
239,Lima,-12.0432,-77.0282,75.90,72,0,3.00,PE,1643579707
326,Vallenar,-28.5708,-70.7581,75.74,45,0,7.83,CL,1643580270
430,Hambantota,6.1241,81.1185,75.54,100,0,2.30,LK,1643580330
453,Ojinaga,29.5667,-104.4167,73.58,32,0,3.44,MX,1643580345
475,Beroroha,-21.6667,45.1667,77.58,61,0,6.11,MG,1643580369
488,Rancho Palos Verdes,33.7445,-118.3870,77.94,51,0,5.01,US,1643580376
569,Sur,22.5667,59.5289,71.37,65,0,5.23,OM,1643580247


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
#hotel df 
hotel_df = narrow_city_df.loc[:,["City", "Country", "Lat", "Lng"]]

#add Hotel Name Column 
hotel_df["Hotel Name"] = ""

#print
hotel_df

,City,Country,Lat,Lng,Hotel Name
239,Lima,PE,-12.0432,-77.0282,
326,Vallenar,CL,-28.5708,-70.7581,
430,Hambantota,LK,6.1241,81.1185,
453,Ojinaga,MX,29.5667,-104.4167,
475,Beroroha,MG,-21.6667,45.1667,
488,Rancho Palos Verdes,US,33.7445,-118.3870,
569,Sur,OM,22.5667,59.5289,


In [9]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [10]:
for index, row in hotel_df.iterrows():
    # get city name, lat, lnt from df
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    # add keyword to params dict
    params["location"] = f"{lat},{lng}"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # results
    results = response['results']
    
    # save the hotel name to dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")
    
    # Wait 1 sec to make another api request to avoid SSL Error
    time.sleep(1)

# Print end of search once searching is completed
print("-------End of Search-------")

Retrieving Results for Index 239: Lima.
Closest hotel in Lima is Sheraton Lima Hotel & Convention Center.
------------
Retrieving Results for Index 326: Vallenar.
Closest hotel in Vallenar is Hotel Del Marques.
------------
Retrieving Results for Index 430: Hambantota.
Closest hotel in Hambantota is The Peacock Beach Hotel Hambantota.
------------
Retrieving Results for Index 453: Ojinaga.
Closest hotel in Ojinaga is Riata Inn Presidio.
------------
Retrieving Results for Index 475: Beroroha.
Missing field/result... skipping.
------------
Retrieving Results for Index 488: Rancho Palos Verdes.
Missing field/result... skipping.
------------
Retrieving Results for Index 569: Sur.
Closest hotel in Sur is Sur Plaza Hotel.
------------
-------End of Search-------


In [11]:
#print df
hotel_df

,City,Country,Lat,Lng,Hotel Name
239,Lima,PE,-12.0432,-77.0282,Sheraton Lima Hotel & Convention Center
326,Vallenar,CL,-28.5708,-70.7581,Hotel Del Marques
430,Hambantota,LK,6.1241,81.1185,The Peacock Beach Hotel Hambantota
453,Ojinaga,MX,29.5667,-104.4167,Riata Inn Presidio
475,Beroroha,MG,-21.6667,45.1667,
488,Rancho Palos Verdes,US,33.7445,-118.3870,
569,Sur,OM,22.5667,59.5289,Sur Plaza Hotel


In [12]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [13]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add the layer to the map
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))